## 네이버 개인 Email로 메일 보내기

### 네이버 메일 설정
- 네이버 메일에서 환경설정 > POP3/SMTP 활성화
- 네이버에서 사용하는 2차 OTP 인증을 사용할 경우에는 에러가 발생하니 유의할 것

### SMTP protocol
[설명 바로가기](https://docs.python.org/2/library/smtplib.html)

파이썬에서 내장된 모듈 smtplib를 사용한다.
위 코드에서보면 sever.ehlo()의 ehlo메서드를 사용하게 되는데 메일을 보낼 때 초기 연결 신호에서
이상이 있는지 없는지를 확인하기 위한 일종의 검수 모듈 이라고 생각.

## Plain Text Type으로 보내기

In [1]:
import smtplib
from email.message import EmailMessage
import getpass

# 비밀번호 암호화 모듈
password = getpass.getpass('Password: ') 

message = EmailMessage()
message['Subject'] = '안녕하세요'
message['From'] = 'sang93423@naver.com'
# 복수 지정 가능
message['To'] = 'orca9sg@gmail.com'
message.set_content('''이번 한달간 놀라운 할인 혜택을 만나보세요~

참여 조건은 일절 없습니다. 그냥 메일만 클릭해보세요

감사합니다.
''')

with smtplib.SMTP_SSL('smtp.naver.com', 465) as server:
    server.ehlo()
    server.login('sang93423', password)
    server.send_message(message)

print('이메일 발송이 완료 되었습니다.')

Password: ········
이메일 발송이 완료 되었습니다.


### 파일 첨부해서 메일 보내기

In [3]:
import os
import smtplib
from email.message import EmailMessage
from email.mime.application import MIMEApplication
import getpass

password = getpass.getpass('Password: ')
message = EmailMessage()
message['Subject'] = '안녕하세요'
message['From'] = 'sang93423@naver.com'
message['To'] = 'orca9sg@gmail.com'
message.set_content('''이번 한달간 놀라운 할인 혜택을 만나보세요.

참여 조건은 일절 없습니다. 그냥 메일만 클릭해보세요.

감사합니다.
''')
message.add_alternative('''
    <h1>노올라운 혜택</h1>
    
    <ul>
        <li> 첫번째 싸다 </li>
        <li> 두번째 빠르다 </li>
    </ul>
''', subtype='html')
file_path = './123.png'
with open(file_path, 'rb') as f:
    filename = os.path.basename(file_path)
    part = MIMEApplication(f.read(), name=filename)
    message.attach(part)

with smtplib.SMTP_SSL ('smtp.naver.com', 465) as server:
    server.ehlo()
    server.login('sang93423', password)
    server.send_message(message)
    
print('이메일 발송이 완료 되었습니다.')

Password: ········
이메일 발송이 완료 되었습니다.


### for loop문을 사용하여 복수의 이미지 파일 발송하기

In [6]:
import os
import smtplib
from email.message import EmailMessage
from email.mime.application import MIMEApplication

import getpass
password = getpass.getpass('Password: ') # 비밀번호 암호화 모듈

message = EmailMessage()
message['Subject'] = '안녕하세요'
message['From'] = 'sang93423@naver.com'
message['To'] = 'orca9sg@gmail.com' # 복수 지정 가능
message.set_content('''이번 한달간 놀라운 할인 혜택을 만나보세요.

참여 조건은 일절 없습니다. 그냥 메일만 클릭해보세요.

감사합니다.
''')

message.add_alternative('''
    <h1>노올라운 혜택</h1>
    
    <ul>
        <li> 첫번째 싸다</li>
        <li> 두번째 빠르다</li>
    </ul>

''', subtype='html')

# for loop 를 사용하여 파일을 순회하여 메시지 보내기
file_path_list = ['./123.png', './456.jpeg']
for file_path in file_path_list:
    with open(file_path, 'rb') as f:
        filename = os.path.basename(file_path)
        part = MIMEApplication(f.read(), name=filename)
        message.attach(part)

with smtplib.SMTP_SSL('smtp.naver.com', 465) as server:
    server.ehlo()
    server.login('sang93423', password)
    server.send_message(message)

print('이메일을 발송했습니다')

Password: ········
이메일을 발송했습니다


### 첨부한 이미지 참조하여 메일 보내기(feat: html의 cid tag)

In [7]:
import os
import smtplib
from email.message import EmailMessage
from email.mime.application import MIMEApplication

import getpass
password = getpass.getpass('Password: ') # 비밀번호 암호화 모듈

message = EmailMessage()
message['Subject'] = '안녕하세요'
message['From'] = 'sang93423@naver.com'
message['To'] = 'orca9sg@gmail.com' # 복수 지정 가능
message.set_content('''이번 한달간 놀라운 할인 혜택을 만나보세요.

참여 조건은 일절 없습니다. 그냥 메일만 클릭해보세요.

감사합니다.
''')

message.add_alternative('''
    <h1>노올라운 혜택</h1>
    
    <img src="cid:123.png" />
    
    <p>이 부분에 이메일 내용을 작성, HTML은 불가<p>

''', subtype='html')

# for loop 를 사용하여 파일을 순회하여 메시지 보내기
file_path_list = ['./123.png', './456.jpeg']
for file_path in file_path_list:
    with open(file_path, 'rb') as f:
        filename = os.path.basename(file_path)
        cid = filename
        img_data = f.read()
        part = MIMEApplication(img_data, name=filename)
        part.add_header('Content-ID', '<' + cid + '>')
        message.attach(part)

with smtplib.SMTP_SSL('smtp.naver.com', 465) as server:
    server.ehlo()
    server.login('sang93423', password)
    server.send_message(message)
    
print('이메일을 발송했습니다.')

Password: ········
이메일을 발송했습니다.
